# Dans ce Mini-projet, nous allons créer un Robot-chat relativement simple qui sera utilisé pour répondre aux questions fréquemment posées.

#Installation des packages
Nous utiliserons simplement pip pour installer ce qui suit:
- numpy
- nltk
- tensorflow
- tflearn

#Données d'entraînement
Comme il s'agit d'un simple chat, on va créer un fichier .JSON qui contient les questions et les responses qu'on va utiliser pour le test. on va appeer notre fichier "intents.json" dans notre projet
#Un extrait du fichier intent.json:

In [ ]:
#{"intents": [
#        {"tag": "greeting",
 #        "patterns": ["Hi", "How are you", "Is anyone there?", "Hello", "Good day", "Whats up"],
 #        "responses": ["Hello!", "Good to see you again!", "Hi there, how can I help?"],
   #      "context_set": ""
    #    },

Ce que nous faisons avec le fichier JSON, c'est créer un ensemble de messages que l'utilisateur est susceptible de saisir et de les mapper à un groupe de réponses appropriées. La balise de chaque dictionnaire du fichier indique le groupe auquel appartient également chaque message. Avec ces données, nous formerons un réseau de neurones pour prendre une phrase de mots et la classer comme l'une des balises de notre fichier. Ensuite, nous pouvons simplement prendre une réponse de ces groupes et l'afficher à l'utilisateur

#1- Importation des bibliothèques Chargement de nos données JSON
Nous allons commencer par importer quelques modules et charger dans nos données json.

In [ ]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy
import tflearn
import tensorflow
import random
import json

with open('C:/Users/Mourad/Documents/NLP/intents.json') as file:
    data = json.load(file)

#2- Extraction de données
Maintenant on va parcourir nos données JSON et d'extraire les données que nous voulons. Pour chaque motif, nous le transformerons en une liste de mots en utilisant "nltk.word_tokenize". Nous ajouterons ensuite chaque modèle dans notre liste docs_words_tag et sa balise associée dans la liste docs_tag

In [ ]:
    words = []
    labels = []
    docs_words_tok = []
    docs_tag = []

    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_words_tok.append(wrds)
            docs_tag.append(intent["tag"])

        if intent["tag"] not in labels:
            labels.append(intent["tag"])

#Racine de mots
Nous utiliserons ce processus de racine des mots (Stemmer) pour réduire le vocabulaire de notre modèle et tenter de trouver le sens plus général derrière les phrases comme on vu dans le cours avec le professseur c'est l'un des algorithmes du Natural language processing.

In [ ]:
    words = [stemmer.stem(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))
    labels = sorted(labels)

#Bag of words
Comme nous le savons, les réseaux de neurones et les algorithmes d'apprentissage automatique nécessitent une entrée numérique. Donc, nous avons besoin d'un moyen de représenter nos phrases avec des nombres et c'est là qu'un sac de mots(bag of words) entre en jeu. Ce que nous allons faire est de représenter chaque phrase avec une liste de la longueur de la quantité de mots dans le vocabulaire de nos modèles. Chaque position dans la liste représentera un mot de notre vocabulaire. Si la position dans la liste est un 1, cela signifie que le mot existe dans notre phrase, s'il s'agit d'un 0, alors le mot n'est pas présent.
En plus de formater notre entrée, nous devons formater notre sortie pour qu'elle ait un sens pour le réseau neuronal.Nous créerons des listes de sortie qui correspondent à la longueur de la quantité des lables/tag que nous avons dans notre ensemble de données. Chaque position dans la liste représentera un label/tag distincte, un 1 dans l'une de ces positions indiquera un label/tag est représentée.

In [ ]:
    training = []
    output = []

    out_empty = [0 for _ in range(len(labels))]

    for idx, doc in enumerate(docs_words_tok):
        bag = []
        
        wrds = [stemmer.stem(w.lower()) for w in doc]

        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)

        output_row = out_empty[:]
        output_row[labels.index(docs_tag[idx])] = 1

        training.append(bag)
        output.append(output_row)
   

    training = numpy.array(training)
    output = numpy.array(output)

#3- Développer d'un modèle
Maintenant que nous avons prétraité toutes nos données, nous sommes prêts à commencer à créer et à entraîner un modèle. Pour nos besoins, nous utiliserons un réseau de neurones à réaction assez standard avec deux couches cachées. Le but de notre réseau sera de regarder un sac de mots et de donner une classe à laquelle ils appartiennent aussi (une de nos balises du fichier JSON).

#3-1 Nous commencerons par définir l'architecture de notre modèle.

In [ ]:
tensorflow.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

modele = tflearn.DNN(net)

#3-2 Entraînement et sauvegarde du modèle
Maintenant nous adapterons nos données au modèle. Le nombre d'epoch que nous avons défini est le nombre de fois où le modèle verra les mêmes informations pendant l'entraînement.
Une fois que nous avons terminé la creation du modèle, nous pouvons l'enregistrer dans le fichier modele.tflearn pour une utilisation dans d'autres scripts.

In [ ]:
modele.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
modele.save("modele.tflearn")

#4- Test du model(prédictions)
Notre processus de génération d'une réponse semblable à ce qui suit:
- Obtenez une entrée de l'utilisateur
- Convertissez-la en un sac de mots
- Obtenez une prédiction du modèle
- Trouvez la classe la plus probable
- Choisissez une réponse de cette classe

In [ ]:
#La fonction bag_of_words transformera notre entrée de chaîne en un sac de mots en utilisant notre liste de mots créée

def bag_of_words(sentence, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(sentence)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for sentc in s_words:
        for indx, word in enumerate(words):
            if word == sentc:
                bag[indx] = 1
            
    return numpy.array(bag)

In [ ]:
#La fonction de chat gérera l'obtention d'une prédiction du modèle et la saisie d'une réponse 
# appropriée à partir de notre fichier de réponses JSON
def chat():
    print("Commencer la conversation avec le Robot (Tapez stop pour arrêter la concersation)!")
    while True:
        entree = input("You: ")
        if entree.lower() == "stop":
            break

        resultat = model.predict([bag_of_words(entree, words)])
        resultat_index = numpy.argmax(resultat)
        tag = labels[resultat_index]

        for tg in data["intents"]:
            if tg['tag'] == tag:
                reponses = tg['responses']

        print(random.choice(reponses))

chat()